In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
import gzip

def parse(path):
  g = gzip.open(path, 'rt', encoding='utf-8')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

review_data = getDF('Data/meta_AMAZON_FASHION.json.gz')
data_set = review_data[:1000]
data_set

,title,image,brand,feature,rank,date,asin,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,[https://images-na.ssl-images-amazon.com/image...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,[https://images-na.ssl-images-amazon.com/image...,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",[https://images-na.ssl-images-amazon.com/image...,Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,[https://images-na.ssl-images-amazon.com/image...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Moby Dick Whale Linked Bracelet,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"26,475,843inClothing,Shoesamp;Jewelry(",925 Sterling,B0006ICJ28,[This sterling silver bracelet brings that fic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,Medium Whale Tail Pendant,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"26,189,330inClothing,Shoesamp;Jewelry(",925 Sterling,B0006ICHEI,[This stunning piece of superior design in ste...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,Diver Holding Pearl Pendant,NaN,NaN,NaN,"26,411,779inClothing,ShoesJewelry(",925 Sterling,B0006ICITW,[A finely detailed SCUBA diver in a perfect po...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,Stylized Celtic Loop,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,"26,187,768inClothing,Shoesamp;Jewelry(",925 Sterling,B0006ICMIY,[Contemporary Celtic style at its best. A well...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
data_set.drop(['feature', 'date', 'description', 'price', 'also_view', 'also_buy', 'fit', 'details', 'similar_item', 'tech1'], axis=1,inplace=True)

In [5]:
data_set.drop(['image', 'rank'],axis=1,inplace=True)

In [6]:
data_set

,title,brand,asin
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),0764443682
1,XCC Qi promise new spider snake preparing men'...,NaN,1291691480
2,Magical Things I Really Do Do Too!,Christopher Manos,1940280001
3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,1940735033
4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,1940967805
...,...,...,...
995,Moby Dick Whale Linked Bracelet,NaN,B0006ICJ28
996,Medium Whale Tail Pendant,NaN,B0006ICHEI
997,Diver Holding Pearl Pendant,NaN,B0006ICITW
998,Stylized Celtic Loop,NaN,B0006ICMIY


In [7]:
data_set.drop(['brand'], axis = 1, inplace=True)

In [8]:
data_set

,title,asin
0,Slime Time Fall Fest [With CDROM and Collector...,0764443682
1,XCC Qi promise new spider snake preparing men'...,1291691480
2,Magical Things I Really Do Do Too!,1940280001
3,"Ashes to Ashes, Oranges to Oranges",1940735033
4,Aether & Empire #1 - 2016 First Printing Comic...,1940967805
...,...,...
995,Moby Dick Whale Linked Bracelet,B0006ICJ28
996,Medium Whale Tail Pendant,B0006ICHEI
997,Diver Holding Pearl Pendant,B0006ICITW
998,Stylized Celtic Loop,B0006ICMIY
